In [1]:
import pandas as pd
import numpy as np
import pandasql as sqldf
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import sys,os
from sklearn.model_selection import train_test_split
import random

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [67]:
#SPEZIELLE VERSION, nicht kopieren!
def hot_encode_top_ohnelapped (column, df, feat_count = 10):
    '''
    hot one encoding, limitiert auf die feat_count häufigsten features
    eines nominalen features um zu Hohe dimensionen zu vermeiden
    
    column: liste mit einem oder mehr Spaltennamen, die hot encoded werden sollen
    df: dataframe der die Datenbasis darstellt
    feat_count: Anzahl Spalten die für jede Spalte encoded werden
    '''
    df_ = df.copy(deep = True)
    
    for col in column:
        
        
        #nur die häufigsten feat_count Featues werden encoded
        encode_features = [x for x in df_[col].value_counts(ascending = False).head(feat_count).index]
        if col == 'status_clean':
            #encode_features = ['lapped', 'Finished', 'DNF']
            encode_features = ['Finished', 'DNF']
        for feature in encode_features:
            col_feature = col + '_'+str(feature)
            if col == 'status_clean' and feature == 'DNF':
                df_[col_feature] = df_.where(np.logical_or(df_[col] == feature,df_[col] == 'lapped'), other = 0)[col]
                df_[col_feature].replace(feature, 1, inplace = True)
                df_[col_feature].replace('lapped', 1, inplace = True)
            else:
                #dort wo feature nicht dem encode feature entspricht wird eine 0 gesetzt
                df_[col_feature] = df_.where(df_[col] == feature, other = 0)[col]
                #encode feature selbst wird in dataframe durch eine 1 ersetzt
                df_[col_feature].replace(feature, 1, inplace = True)
            
        
        #löschen der nun "bereinigten" Spalte
        del df_[col]
        
    return df_

In [85]:
def hot_encode_top (column, df, feat_count = 10):
    '''
    hot one encoding, limitiert auf die feat_count häufigsten features
    eines nominalen features um zu Hohe dimensionen zu vermeiden
    
    column: liste mit einem oder mehr Spaltennamen, die hot encoded werden sollen
    df: dataframe der die Datenbasis darstellt
    feat_count: Anzahl Spalten die für jede Spalte encoded werden
    '''
    df_ = df.copy(deep = True)
    
    for col in column:
        
        
        #nur die häufigsten feat_count Featues werden encoded
        encode_features = [x for x in df_[col].value_counts(ascending = False).head(feat_count).index]
        if col == 'status_clean':
            #encode_features = ['lapped', 'Finished', 'DNF']
            encode_features = ['Finished', 'DNF']
        for feature in encode_features:
            col_feature = col + '_'+str(feature)
            
            #dort wo feature nicht dem encode feature entspricht wird eine 0 gesetzt
            df_[col_feature] = df_.where(df_[col] == feature, other = 0)[col]
            #encode feature selbst wird in dataframe durch eine 1 ersetzt
            df_[col_feature].replace(feature, 1, inplace = True)
            
        
        #löschen der nun "bereinigten" Spalte
        del df_[col]
        
    return df_

In [101]:
rain_id = [847,861,879,910,914,934,942,953,957,967,970,982]
if os.path.exists('sliced_data70'):
    csv_filenames = []
    #auslesen aller csv file dateinamen aus formula 1 datensatz und abspeichern in liste
    for filename in os.listdir(os.getcwd()+'/sliced_data70'):
        typ = filename.split('.')[-1]
        name = filename.split('.')[0]
        if typ == 'csv':
            csv_filenames.append(filename)
    sliced_races = {}
    #einlesen und abspeichern als dataframe aller dateien
    for file in csv_filenames:
        try:
            df = pd.read_csv('sliced_data70/'+file, engine = 'python', sep = ';', decimal = '.')
            del df['Unnamed: 0']
        except Exception as e:
            df = pd.read_csv('sliced_data70/'+file, engine = 'c', sep = ';', decimal = '.')
            del df['Unnamed: 0']
            print(e)
        #print(df.head())
        f = int(file.split('_')[-1].split('.')[0])
        df["rain"] = 0
        if list(df["raceId"])[0] in rain_id:
            df["rain"] = 1
        sliced_races[f] = df
    print('Einlesen der sliced Dateien erfolgreich')
else:
    print('Dateien können nicht eingelesen werden, da kein entsprechendes Verzeichnis existiert!')
    
if os.path.exists('split_data'):
    csv_filenames = []
    #auslesen aller csv file dateinamen aus formula 1 datensatz und abspeichern in liste
    for filename in os.listdir(os.getcwd()+'/split_data'):
        typ = filename.split('.')[-1]
        name = filename.split('.')[0]
        if typ == 'csv':
            csv_filenames.append(filename)
    split_by_race = {}
    #einlesen und abspeichern als dataframe aller dateien
    for file in csv_filenames:
        try:
            df = pd.read_csv('split_data/'+file, engine = 'python', sep = ';', decimal = '.')
            del df['Unnamed: 0']
        except Exception as e:
            df = pd.read_csv('split_data/'+file, engine = 'c', sep = ';', decimal = '.')
            del df['Unnamed: 0']
            print(e)
        #print(df.head())
        f = int(file.split('_')[-1].split('.')[0])
        split_by_race[f] = df
    print('Einlesen der split Dateien erfolgreich')
else:
    print('Dateien können nicht eingelesen werden, da kein entsprechendes Verzeichnis existiert!')

Einlesen der sliced Dateien erfolgreich
Einlesen der split Dateien erfolgreich


In [3]:
if os.path.exists('sliced_data70'):
    csv_filenames = []
    #auslesen aller csv file dateinamen aus formula 1 datensatz und abspeichern in liste
    for filename in os.listdir(os.getcwd()+'/sliced_data70'):
        typ = filename.split('.')[-1]
        name = filename.split('.')[0]
        if typ == 'csv':
            csv_filenames.append(filename)
    sliced_races = {}
    #einlesen und abspeichern als dataframe aller dateien
    for file in csv_filenames:
        try:
            df = pd.read_csv('sliced_data70/'+file, engine = 'python', sep = ';', decimal = '.')
            del df['Unnamed: 0']
        except Exception as e:
            df = pd.read_csv('sliced_data70/'+file, engine = 'c', sep = ';', decimal = '.')
            del df['Unnamed: 0']
            print(e)
        #print(df.head())
        f = int(file.split('_')[-1].split('.')[0])
        sliced_races[f] = df
    print('Einlesen der sliced Dateien erfolgreich')
else:
    print('Dateien können nicht eingelesen werden, da kein entsprechendes Verzeichnis existiert!')
    
if os.path.exists('split_data'):
    csv_filenames = []
    #auslesen aller csv file dateinamen aus formula 1 datensatz und abspeichern in liste
    for filename in os.listdir(os.getcwd()+'/split_data'):
        typ = filename.split('.')[-1]
        name = filename.split('.')[0]
        if typ == 'csv':
            csv_filenames.append(filename)
    split_by_race = {}
    #einlesen und abspeichern als dataframe aller dateien
    for file in csv_filenames:
        try:
            df = pd.read_csv('split_data/'+file, engine = 'python', sep = ';', decimal = '.')
            del df['Unnamed: 0']
        except Exception as e:
            df = pd.read_csv('split_data/'+file, engine = 'c', sep = ';', decimal = '.')
            del df['Unnamed: 0']
            print(e)
        #print(df.head())
        f = int(file.split('_')[-1].split('.')[0])
        split_by_race[f] = df
    print('Einlesen der split Dateien erfolgreich')
else:
    print('Dateien können nicht eingelesen werden, da kein entsprechendes Verzeichnis existiert!')

Einlesen der sliced Dateien erfolgreich
Einlesen der split Dateien erfolgreich


In [110]:
keys = list(sliced_races.keys())
random.shuffle(keys)
sliced_shuffled = {}
for key in keys:
    sliced_shuffled[key] = sliced_races[key]


#columns werden definiert, die erst einmal nicht beachtet werden sollen
nogo_columns = [#'grid',
                #'race_completion',
                'lap_position','circuitId','lap_number',
                'podium_position', 'raceId',
                'grandprix_name', 'driver_fullname',
               'constructor_name', 'total_laps',
               'status_clean', 'constructorId',
                'total_milliseconds', 'driverId'
               'lap_in_milliseconds','year', 'stop_binary']

#definieren eines linearen Regressionsmodells
reg = LinearRegression()

train_break = 120
category_count = 13
counter = 0

print('---training---')
full_races = pd.DataFrame()
for raceId, race in sliced_shuffled.items():
    full = pd.DataFrame(columns = race.columns)
    if counter == train_break:
        break
    else:
        last_lap = max(race.lap_number.tolist())
        for did in race.driverId.unique():
            stop_sum = np.sum(race.where(race.driverId == did).dropna(how='all')['stop_binary'])
            race['stop_sum'] = stop_sum
            driver_df = race.where(np.logical_and(race.lap_number == last_lap, race.driverId == did)).dropna(how = 'all')
            full = full.append(driver_df)
        #for elem in full['driverId'].unique():
        #    full[str(elem)] = full['driverId'] == elem
        #for elem in full['constructorId'].unique():
        #    full[str(elem)] = full['constructorId'] == elem
        #for c in ['driverId','constructorId']:
        #temp = pd.get_dummies(data = full, columns = ['driverId','constructorId'], drop_first = True)
        #temp.replace(np.nan,0,inplace = True)
        #full = temp.copy(deep = True)
        ##for col in temp.columns:
        #    full[col] = temp[col]
        full_races = full_races.append(full)
#full_races = full_races.replace(np.nan,0)   
#full_races = hot_encode_top(['status_clean','driverId', 'constructorId'],full_races,15)
full_races = hot_encode_top(['status_clean','driverId'],full_races,category_count)
cols = [col for col in full_races.columns if col not in nogo_columns] 

#umrechnen in minuten
y = np.array(full_races['total_milliseconds']/60000)
races = full_races[cols]  
#races = races.dropna(how = 'any')
X = np.array(races)
reg = reg.fit(X, y)
A = reg.predict(X)
mse = mean_squared_error(A, y)
print('mse:', mse)

counter = 0

full_races = pd.DataFrame()
print('---testing---')
for raceId, race in sliced_shuffled.items():
    full = pd.DataFrame(columns = race.columns)
    if counter == train_break:
        last_lap = max(race.lap_number.tolist())
        for did in race.driverId.unique():
            stop_sum = np.sum(race.where(race.driverId == did).dropna(how='all')['stop_binary'])
            race['stop_sum'] = stop_sum
            driver_df = race.where(np.logical_and(race.lap_number == last_lap, race.driverId == did)).dropna(how = 'all')
            full = full.append(driver_df)
            
        #temp = pd.get_dummies(data = full, columns = ['driverId','constructorId'], drop_first = True)
        #temp.replace(np.nan,0,inplace = True)
        #full = temp.copy(deep = True)
        #for col in temp.columns:
        #    full[col] = temp[col]
        #full = hot_encode_top(['status_clean'],full)
        #full = hot_encode_top(['status_clean','driverId', 'constructorId'],full)
        full_races = full_races.append(full)
        
    else:
        counter += 1
#full_races = hot_encode_top(['status_clean','driverId', 'constructorId'],full_races,15)
full_races = hot_encode_top(['status_clean','driverId'],full_races,category_count)
full_races = full_races.replace(np.nan,0)   
y = np.array(full_races['total_milliseconds']/60000)
cols = [col for col in full_races.columns if col not in nogo_columns] 
races = full_races[cols]  
X = np.array(races)
A = reg.predict(X)
mse = mean_squared_error(A, y)
print('mse:', mse)

#hinzufügen der 'wichtigen' spalten zur späteren auswertung der vorhersagen auf dem trainingsdatensatz
full_races['prediction'] = A
full_races['total_milliseconds'] = y
full_races['total_minutes'] = y/60000

---training---
mse: 17.29996546832689
---testing---
mse: 12.965065493605882


In [111]:
treffer = 0
nahdran = 0
weiterweg = 0
for raceId in full_races.raceId.unique():
    
    race = full_races.where(full_races.raceId == raceId).dropna(how = 'all')
    winner = race.where(race.podium_position == 1).dropna(how = 'all')['driver_fullname'].tolist()[0]
    second = race.where(race.podium_position == 2).dropna(how = 'all')['driver_fullname'].tolist()[0]
    third = race.where(race.podium_position == 3).dropna(how = 'all')['driver_fullname'].tolist()[0]
    
    fourth = race.where(race.podium_position == 4).dropna(how = 'all')['driver_fullname'].tolist()[0]
    fifth = race.where(race.podium_position == 5).dropna(how = 'all')['driver_fullname'].tolist()[0]
    sixth = race.where(race.podium_position == 6).dropna(how = 'all')['driver_fullname'].tolist()[0]
    seventh = race.where(race.podium_position == 7).dropna(how = 'all')['driver_fullname'].tolist()[0]
    
    pred_winner = race.where(race.prediction == min(race.prediction)).dropna(how = 'all')['driver_fullname'].tolist()[0]
    
    podium = [second,third]
    seven = [fourth,fifth,sixth,seventh]
    if pred_winner == winner:
        treffer += 1
    if pred_winner in podium:
        nahdran += 1
    if pred_winner in seven:
        weiterweg += 1
    print('winner, second, third:', winner,',' ,second,',' , third)
    print('predicted winner:', pred_winner,'\n')
    
print('genaue treffer:', treffer)
print('nah dran:', nahdran)
print('weiter weg:', weiterweg)

winner, second, third: Lewis Hamilton , Sebastian Vettel , Sergio PÃŒÂ©rez
predicted winner: Valtteri Bottas 

winner, second, third: Nico Rosberg , Lewis Hamilton , Sebastian Vettel
predicted winner: Nico Rosberg 

winner, second, third: Lewis Hamilton , Nico Rosberg , Sebastian Vettel
predicted winner: Kimi RÃŒ_ikkÃŒÂ¦nen 

winner, second, third: Sebastian Vettel , Jenson Button , Fernando Alonso
predicted winner: Fernando Alonso 

winner, second, third: Nico Rosberg , Sebastian Vettel , Lewis Hamilton
predicted winner: Max Verstappen 

winner, second, third: Nico Rosberg , Lewis Hamilton , Valtteri Bottas
predicted winner: Sebastian Vettel 

winner, second, third: Lewis Hamilton , Nico Rosberg , Fernando Alonso
predicted winner: Nico Rosberg 

winner, second, third: Lewis Hamilton , Valtteri Bottas , Kimi RÃŒ_ikkÃŒÂ¦nen
predicted winner: Lewis Hamilton 

winner, second, third: Sebastian Vettel , Fernando Alonso , Lewis Hamilton
predicted winner: Esteban GutiÃŒÂ©rrez 

winner, second

In [105]:
races.head()

,lap_in_milliseconds,race_completion,grid,form,sum_milliseconds_pro_lap,rain,stop_sum,status_clean_Finished,status_clean_DNF,driverId_815.0,...,driverId_1.0,driverId_817.0,driverId_20.0,driverId_13.0,driverId_3.0,driverId_154.0,driverId_4.0,driverId_18.0,driverId_822.0,driverId_814.0
45,81230.0,0.686567,13.0,15.00,3812800.0,0.0,2.0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
91,84766.0,0.686567,12.0,12.75,3813516.0,0.0,3.0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
173,81048.0,0.686567,7.0,9.50,3798965.0,0.0,2.0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
219,81987.0,0.686567,16.0,15.00,3840887.0,0.0,2.0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
265,97800.0,0.686567,14.0,15.25,3904590.0,0.0,3.0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [106]:
races.columns

Index(['lap_in_milliseconds', 'race_completion', 'grid', 'form',
       'sum_milliseconds_pro_lap', 'rain', 'stop_sum', 'status_clean_Finished',
       'status_clean_DNF', 'driverId_815.0', 'driverId_807.0', 'driverId_8.0',
       'driverId_1.0', 'driverId_817.0', 'driverId_20.0', 'driverId_13.0',
       'driverId_3.0', 'driverId_154.0', 'driverId_4.0', 'driverId_18.0',
       'driverId_822.0', 'driverId_814.0'],
      dtype='object')